# Homework 4
#### Author: Darren Colby
#### Course: COSC 74
#### Date: March 10th, 2022

## Setting up the notebook for subsequent questions

In [1]:
# Imports
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Load the data
naive = np.loadtxt("hw4_naive.csv", delimiter = ",", skiprows=1)

## Don't be so naive!
These questions are all related to implementing and using the Naive Bayes classifier.

In [3]:
# Identify features and labels
x = naive[:, :-1]
y = naive[:, -1]

# Create train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=17)

### Fitting the model
The first step here is fitting our model. Unlike other supervised learning methods that rely on gradient descent to learn some weights that minimize a cost function, Naive Bayes is a probabilistic model. This means the only thing we need to do in this step is calculate the priors for each class label.

In [4]:
def fit(y_train):
    """Calculates the prior for each class to use in a Bayesian framework
       ------------------------------------------------------------------
       
       Parameters:
           y_train: a 1D array of classes to predict from the training data
        
       Returns a dictionary where keys are classes and values are their priors"""
    # The unique classes in the dataset
    labels = np.unique(y_train)
    
    # Each class is a key and its prior is the value
    priors_dict = {label:np.log(y_train[y_train == label].size / y_train.size) for label in labels}
    
    return priors_dict

### Calculating the likelihood
In Naive Bayes we need to calculate the likelihood that the value of each feature in each observation of our test dataset came from the same feature in the training data set conditional on its class label. In real life this involves multiplying the probabilities that each value in the test data set came from each feature conditional on each class label in the traiing data set. However, to avoid floating point precision errors, we add the log of each of these probabilities, which gives us the same predicted classes. Since calculating these probabilities depends on the probability distribution of the features in our traiing data, we need to use the probability density or probability mass function for each probability distribution we assume our data was generated from. For our purposes, we assume features are either multinomially or Gaussian distributed and therefore define functions to calculate the Gaussian and multinomial log-likelihoods.

In [5]:
def multinomial_log_likelihood(row, subsetted_data, smoothing, eps):
    """Calculates and sums the log of posteriors for each feature in a row of data in a numpy array from the Gausssian PDF
       -------------------------------------------------------------------------------------------------------------------
       
       Parameters:
           row: the row of data to calculate the multinomial log-likelihood for
           subsetted_data: a 2D numpy array of data subsetted to a class of interest
           smoothing: a smoothing parameter for Laplace smoothing
           eps: a constant to add to probabilities before taking their log to avoid dividing by zero
           
       Returns the multinomical log-likelihood for the given row of data"""
    
    # The sum of the log probabilities for each feature
    total_log_prob = 0
    
    # Loop through each feature in each data point
    for num in row:
       
        # Get the index of the array, which corresponds to the column to use for the multinomial PDF
        idx = np.where(row == num)
        
        # Find the number of observations with the same value and all values in the training data subsetted by class
        same_obs = np.where(subsetted_data[:, idx] == num)[0].size + smoothing
        number_obs = subsetted_data[:, idx].size + subsetted_data.shape[1]
        
        # Update the total log likelihood
        total_log_prob += np.log((same_obs / number_obs) + eps)
        
    return total_log_prob

In [6]:
def gaussian_log_likelihood(row, subsetted_data, eps):
    """Calculates and sums the log of posteriors for each feature in a row of data in a numpy array from the Gausssian PDF
       ------------------------------------------------------------------------------------------------------------------
       
       Parameters:
           row: the row of data to calculate the gaussian log-likelihood for
           subsetted_data: a 2D numpy array of data subsetted to a class of interest
           eps: a constant to add to probabilities before taking their log to avoid dividing by zero
           
        Returns the Gaussian log-likelihood for the given row of data"""
    
    # The sum of log probabilities for each feature
    total_log_prob = 0
    
    # Loop through each feature of the data point
    for num in row:
        
        # Get the index in the array. This corresponds to the column to get the mean and standard deviation of.
        idx = np.where(row == num)
        
        # Parameters of the normal distribution in the column corresponding to the index 
        mean = np.mean(subsetted_data[:, idx])
        std_dev = np.std(subsetted_data[:, idx])
        
        # Add the log of the probability from the Gaussian PDF
        total_log_prob += np.log(((1.0 / np.sqrt(2.0 * np.pi * std_dev**2)) * np.exp(-((num - mean)**2) / \
                                                                                     (2 * (std_dev**2)))) + eps)
    
    return total_log_prob

### Making predictions
To make predictions, we need to go through each data point and calculate the log-likelihood that each of its feature values came from the feature in the training data. We do this with each class label by subsetting the data to observations whose class label is the class we are iterating through and summing them up. Since we are using a Bayesian framework, we also add the log of the prior to each observation for each class. The code below does this, however, it only uses one for loop because it takes advantage of NumPy's ability to apply a function to an entire row or column and broadcast when adding a scalar to a vector.

In [7]:
def predict(x_train, y_train, x_test, pdf="gaussian", smoothing=1.0, eps=1e-9):
    """Predicts a Naive Bayes classifier
       ---------------------------------
       
       Parameters:
           x_train: input features for the training data set
           y_train: class labels for the training data set
           x_test: input features from the test data set to predict the labels with
           pdf: the probability density fuction to use to calculate the likelihood; current options are guassian and multinomial
           smoothing: a smoothing parameter for Laplace smoothing; if pdf is gaussian this prameter is ignored
           eps: a constant to add to probabilities before taking their log to avoid dividing by zero
           
        Returns a tuple of predicted class labels and the log posteriors for each class and data point"""
    
    # Safety check
    if pdf not in ["multinomial", "gaussian"]:
        raise ValueError("This Naive Bayes implementation currently only supports multinomial and Gaussian distributions")
    
    # Store the unique labels
    labels = np.unique(y_train)
    
    # Get the priors
    priors = fit(y_train)
    
    # Stores posteriors
    posteriors_list = []
    
    # Stores predictions
    predictions = []
    
    # Loop through each unique class
    for label in labels:
        
        # Data subsetted to each label
        subset = x_train[y_train == label]
        
        # Applies the log likelihood to every element in every row of the test data and calculates its posterior
        if pdf == "gaussian":
            posteriors = (priors[label] + np.apply_along_axis(gaussian_log_likelihood, 1, x_test, subset, eps)).tolist()
        
        if pdf == "multinomial":
            posteriors = (priors[label] + np.apply_along_axis(multinomial_log_likelihood, 1, x_test, subset, 
                                                              smoothing, eps)).tolist()
            
        posteriors_list.append(posteriors)
        
    # Loop through the posteriors to find the class with the highest posterior
    for idx in np.apply_along_axis(np.argmax, 1, np.array(posteriors_list).T):
        predictions.append(int(labels[idx]))
        
    return np.array(predictions), np.array(posteriors_list).T

### Evaluating the model
No matter how complicated a model is, if it does not perform well, there is no sense in using it. Therfore, it would be useful to know some metrics like the number of true positives, true negatives, false positives, false negatives, accuracy, precision, recall, and F1 score, which the function below calculates.

In [8]:
def evaluate(y_actual, y_pred):
    """Calculates true positives, true negatives, false positives, false negatives, accuracy, precision, recall, and F1 score
       ----------------------------------------------------------------------------------------------------------------------
       
       Parameters:
            y_actual: actual labels
            y_pred: predicted labels
            
       Returns a tuple of the true positives, true negatives, false positives, false negatives, accuracy, precision, recall, 
           and F1 score"""
    
    # Store the class labels
    labels = np.unique(y_actual)
    
    # When there are only two class labels
    if y_actual.size == 2:
        
        # Calculate positives and negatives
        true_positive = np.sum(np.where((y_pred == 1) & (y_actual == 1)))
        true_negative = np.sum(np.where((y_pred == 0) & (y_actual == 0)))
        false_positive = np.sum(np.where((y_pred == 1) & (y_actual == 0)))
        false_negative = np.sum(np.where((y_pred == 0) & (y_actual == 1)))
    
    # When doing multiclass classification
    else:
        
        for label in labels:
        
            # Calculate positives and negatives
            true_positive = np.sum(np.where((y_pred == label) & (y_actual == label)))
            true_negative = np.sum(np.where((y_pred != label) & (y_actual != label)))
            false_positive = np.sum(np.where((y_pred == label) & (y_actual != label)))
            false_negative = np.sum(np.where((y_pred != label) & (y_actual == label)))
            
    # Calculate accuracy
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    
    # Calculate precision and recall
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    
    # Calculate the F1 score
    f1 = (2 * (precision * recall)) / (precision + recall)
        
    return true_positive, true_negative, false_positive, false_negative, accuracy, precision, recall, f1

### Using the model
Now it is time to use the model. In the first cell, we assume a multinomial distribution to train, predict, ad evaluate our model. The second cell assumes a gaussian distribution.

In [9]:
# Multinomial distribution

# Train and predict
class_predictions1, posterior_predictions1 = predict(x_train, y_train, x_test, pdf="multinomial")

# Evaluate
evaluations1 = evaluate(y_test, class_predictions1)
print("The F1 score with a multiomial distribution is " + str(evaluations1[7]))

The F1 score with a multiomial distribution is 0.8505722541329463


In [10]:
# Gaussian distribution

# Train and predict
class_predictions2, posterior_predictions2 = predict(x_train, y_train, x_test, pdf="gaussian")

# Evaluate
evaluations2 = evaluate(y_test, class_predictions2)
print("The F1 score with a Gaussian distribution is " + str(evaluations2[7]))

The F1 score with a Gaussian distribution is 0.31365711284270037
